In [2]:
# Parameters
DATE = 20211102
N_TRIALS = 50
NN_MAX_EPOCHS = 100
CNN_MAX_EPOCHS = 200
SCORING_METRIC = "f1_macro"

# Multilabel Model Training

In [3]:
import os
import pickle

import bento
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from optuna.integration import SkorchPruningCallback
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, StandardScaler
from sklearn.svm import SVC
from skorch import NeuralNet, NeuralNetClassifier
from skorch.callbacks import EarlyStopping, EpochScoring
from xgboost import XGBRFClassifier

In [4]:
# Other general variables
model_dir = f"../../models/ml_multilabel_{DATE}"

# Make folder to store study and model
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

## Cell x Feature Matrix

In [5]:
data = bento.io.read_h5ad("../../data/locfish/locfish_eval_20211019.h5ad")
data.shape

(10000, 1)

In [6]:
# List of features
features = [
    "cell_inner_proximity",
    "nucleus_inner_proximity",
    "nucleus_outer_proximity",
    "cell_inner_asymmetry",
    "nucleus_inner_asymmetry",
    "nucleus_outer_asymmetry",
    "l_max",
    "l_max_gradient",
    "l_min_gradient",
    "l_monotony",
    "l_half_radius",
    "point_dispersion",
    "nucleus_dispersion",
]

cell_by_feature = []
for f in features:
    cell_by_feature.append(data.to_df(f))
cell_by_feature = (
    pd.concat(cell_by_feature, axis=1).reset_index(drop=True).astype(float)
)
cell_by_feature.columns = features

## Prepare datasets

In [7]:
classes = ["cell_edge", "cytoplasmic", "none", "nuclear", "nuclear_edge"]

Split train/test features

In [8]:
X = torch.FloatTensor(cell_by_feature.values)

le = LabelBinarizer().fit(classes)
y = le.transform(data.to_df("pattern").values.flatten())
y = torch.LongTensor(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=22, stratify=y
)

Split train/test image features

In [9]:
from torchvision import datasets, transforms

dataset = datasets.ImageFolder(
    "data/locfish/eval_img",
    transform=transforms.Compose([transforms.Grayscale(), transforms.ToTensor()]),
)
X_img = np.array([x[0].numpy() for x in dataset])
y_img = np.array(dataset.targets)

le = LabelBinarizer().fit(dataset.targets)
y_img = le.transform(y_img)

X_img_train, X_img_test, y_img_train, y_img_test = train_test_split(
    X_img, y_img, train_size=0.8, random_state=22, stratify=y_img
)

# Prepare nn architectures

In [10]:
class FCModule(nn.Module):
    def __init__(
        self,
        l0_size,
        l1_size,
    ):
        super().__init__()
        fc_layers = []
        in_features = 13

        # Hidden layers
        for layer_size in [l0_size, l1_size]:
            fc_layers.extend(
                [
                    nn.Linear(in_features, layer_size),
                    nn.BatchNorm1d(layer_size),
                    nn.Dropout(0.2),
                    nn.ReLU(),
                ]
            )

            in_features = layer_size

        # Output layer
        fc_layers.append(nn.Linear(in_features, 2))
        self.model = nn.Sequential(*fc_layers)

    def forward(self, x):
        x = self.model(x.float())
        x = F.softmax(x, dim=-1)
        return x

In [11]:
def get_conv_dim(in_size, padding, dilation, kernel_size, stride):
    outsize = 1 + (in_size + 2 * padding - dilation * (kernel_size - 1) - 1) / stride
    return int(outsize)


class SpotsBinaryModule(nn.Module):
    def __init__(
        self,
        n_conv_layers,
        in_dim,
        out_channels,
        kernel_size,
        f_units_l0,
        f_units_l1,
    ) -> None:
        super().__init__()
        conv_layers = []

        in_channels = 1
        in_dim = in_dim

        # Stack (convolutions + batchnorm + activation) + maxpool
        for i in range(n_conv_layers):
            conv_layers.extend(
                [
                    nn.Conv2d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=kernel_size,
                    ),
                    nn.BatchNorm2d(out_channels),
                    nn.Dropout(0.2),
                    nn.ReLU(),
                ]
            )

            # Compute convolved output dimensions
            in_dim = get_conv_dim(
                in_dim, padding=0, dilation=1, kernel_size=kernel_size, stride=1
            )

            in_channels = out_channels
            out_channels *= 2

        out_channels = int(out_channels / 2)

        conv_layers.append(nn.MaxPool2d(2, 2))
        in_dim = int(in_dim / 2)

        # We optimize the number of layers, hidden units and dropout ratio in each layer.
        fc_layers = [nn.Flatten()]

        # Compute flatten size
        in_features = out_channels * in_dim * in_dim
        for i in [f_units_l0, f_units_l1]:
            out_features = i
            fc_layers.extend(
                [
                    nn.Linear(in_features, out_features),
                    nn.BatchNorm1d(out_features),
                    nn.Dropout(0.2),
                    nn.ReLU(),
                ]
            )

            in_features = out_features

        fc_layers.append(nn.Linear(in_features, 2))
        self.model = torch.nn.Sequential(*[*conv_layers, *fc_layers])

    def forward(self, x):
        x = self.model(x)
        x = F.softmax(x, dim=-1)
        return x

# Optimize model hyperparameters

## Gradient-boosted Random Forest Classifier

In [12]:
def objective(trial):

    clf = OneVsRestClassifier(
        make_pipeline(
            StandardScaler(),
            XGBRFClassifier(
                n_estimators=trial.suggest_int("n_estimators", 10, 500, 10),
                max_depth=trial.suggest_int("max_depth", 2, 20),
                learning_rate=trial.suggest_float("learning_rate", 1e-4, 1, log=True),
                colsample_bytree=trial.suggest_float("colsample_bytree", 0.2, 0.8),
                reg_lambda=trial.suggest_float("reg_lambda", 1e-7, 1e-1, log=True),
                use_label_encoder=False,
                eval_metric="logloss",  # set to hide warning
            ),
        )
    )

    # Avoid overfitting with 4-fold cross validation
    return cross_val_score(
        clf, X_train.numpy(), y_train.numpy(), scoring=SCORING_METRIC, cv=4
    ).mean()

# 3. Create a study object and optimize the objective function.
rf_study = optuna.create_study(
    direction="maximize",
    study_name="xgbrf",
    storage=f"sqlite:///{model_dir}/optuna.db",
    load_if_exists=True,
)
rf_study.optimize(
    objective, n_trials=N_TRIALS, n_jobs=1, show_progress_bar=True, gc_after_trial=False
)

[I 2021-11-02 11:22:01,223] A new study created in RDB with name: xgbrf


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2021-11-02 11:22:11,729] Trial 0 finished with value: 0.8242056679880935 and parameters: {'n_estimators': 190, 'max_depth': 8, 'learning_rate': 0.28621828898249235, 'colsample_bytree': 0.30500507794771564, 'reg_lambda': 6.785891650876217e-05}. Best is trial 0 with value: 0.8242056679880935.
[I 2021-11-02 11:22:46,377] Trial 1 finished with value: 0.8589165005541043 and parameters: {'n_estimators': 420, 'max_depth': 14, 'learning_rate': 0.042374270114706694, 'colsample_bytree': 0.5235112801067936, 'reg_lambda': 0.0006913686764799664}. Best is trial 1 with value: 0.8589165005541043.
[I 2021-11-02 11:23:11,644] Trial 2 finished with value: 0.8613892101064895 and parameters: {'n_estimators': 270, 'max_depth': 10, 'learning_rate': 0.4471771956313932, 'colsample_bytree': 0.6962156199348712, 'reg_lambda': 0.058555999648835386}. Best is trial 2 with value: 0.8613892101064895.
[I 2021-11-02 11:23:24,010] Trial 3 finished with value: 0.8390890840757422 and parameters: {'n_estimators': 370, 'm

## Support Vector Classifier

In [13]:
def objective(trial):

    clf = OneVsRestClassifier(
        make_pipeline(
            StandardScaler(),
            SVC(
                kernel=trial.suggest_categorical(
                    "kernel", ["linear", "poly", "rbf", "sigmoid"]
                ),
                C=trial.suggest_float("C", 1e-5, 1, log=True),
                degree=trial.suggest_int("degree", 2, 5),
                probability=True,
            ),
        )
    )

    # Avoid overfitting with 4-fold cross validation
    return cross_val_score(clf, X_train.numpy(), y_train.numpy(), scoring=SCORING_METRIC, cv=4).mean()

# 3. Create a study object and optimize the objective function.
svc_study = optuna.create_study(
    direction="maximize",
    study_name="svc",
    storage=f"sqlite:///{model_dir}/optuna.db",
    load_if_exists=True,
)
svc_study.optimize(
    objective, n_trials=N_TRIALS, n_jobs=1, show_progress_bar=True, gc_after_trial=False
)

[I 2021-11-02 11:50:30,620] A new study created in RDB with name: svc


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2021-11-02 11:51:41,673] Trial 0 finished with value: 0.4897810466018762 and parameters: {'kernel': 'rbf', 'C': 0.003593047604783355, 'degree': 5}. Best is trial 0 with value: 0.4897810466018762.
[I 2021-11-02 11:52:02,298] Trial 1 finished with value: 0.7100393710035365 and parameters: {'kernel': 'linear', 'C': 0.12595400208696808, 'degree': 5}. Best is trial 1 with value: 0.7100393710035365.
[I 2021-11-02 11:52:45,141] Trial 2 finished with value: 0.43236039786776154 and parameters: {'kernel': 'poly', 'C': 0.022140707257167415, 'degree': 5}. Best is trial 1 with value: 0.7100393710035365.
[I 2021-11-02 11:53:29,104] Trial 3 finished with value: 0.360130905592316 and parameters: {'kernel': 'poly', 'C': 0.019775182176098326, 'degree': 4}. Best is trial 1 with value: 0.7100393710035365.
[I 2021-11-02 11:55:11,754] Trial 4 finished with value: 0.0 and parameters: {'kernel': 'sigmoid', 'C': 0.00027303643445618847, 'degree': 5}. Best is trial 1 with value: 0.7100393710035365.
[I 2021-11

## Feed Forward Neural Net

In [14]:
def objective(trial):

    clf = OneVsRestClassifier(
        NeuralNetClassifier(
            module=FCModule,
            optimizer=torch.optim.Adam,
            module__l0_size=trial.suggest_int("module__l0_size", 10, 300),
            module__l1_size=trial.suggest_int("module__l1_size", 10, 300),
            max_epochs=NN_MAX_EPOCHS,
            # Internal 80/20 train valid split
            device="cuda",
            verbose=False,
        ),
    )

    clf.fit(X_train, y_train)

    # Avoid overfitting by evaluating internal 20% validation
    return np.mean([e.history[-1, "valid_loss"] for e in clf.estimators_])

# 3. Create a study object and optimize the objective function.
nn_study = optuna.create_study(
    direction="minimize",
    study_name="ffnn",
    storage=f"sqlite:///{model_dir}/optuna.db",
    load_if_exists=True,
)
nn_study.optimize(
    objective, n_trials=N_TRIALS, n_jobs=1, show_progress_bar=True, gc_after_trial=False
)

[I 2021-11-02 12:26:31,196] A new study created in RDB with name: ffnn


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2021-11-02 12:27:42,282] Trial 0 finished with value: 0.1332475958019495 and parameters: {'module__l0_size': 11, 'module__l1_size': 293}. Best is trial 0 with value: 0.1332475958019495.
[I 2021-11-02 12:28:46,077] Trial 1 finished with value: 0.12635636632330716 and parameters: {'module__l0_size': 208, 'module__l1_size': 297}. Best is trial 1 with value: 0.12635636632330716.
[I 2021-11-02 12:29:49,099] Trial 2 finished with value: 0.1213867208827287 and parameters: {'module__l0_size': 101, 'module__l1_size': 43}. Best is trial 2 with value: 0.1213867208827287.
[I 2021-11-02 12:30:53,127] Trial 3 finished with value: 0.12290588059648873 and parameters: {'module__l0_size': 165, 'module__l1_size': 35}. Best is trial 2 with value: 0.1213867208827287.
[I 2021-11-02 12:31:57,612] Trial 4 finished with value: 0.1305319029800594 and parameters: {'module__l0_size': 250, 'module__l1_size': 257}. Best is trial 2 with value: 0.1213867208827287.
[I 2021-11-02 12:33:01,224] Trial 5 finished with 

## CNN Neural Net

In [15]:
def objective(trial):

    # Initialize classifier model
    clf = OneVsRestClassifier(
        NeuralNetClassifier(
            module=SpotsBinaryModule,
            module__n_conv_layers=2,
            module__in_dim=64,
            module__out_channels=trial.suggest_int("module__out_channels", 2, 16),
            module__kernel_size=3,
            module__f_units_l0=trial.suggest_int("module__f_units_l0", 10, 300),
            module__f_units_l1=trial.suggest_int("module__f_units_l1", 10, 100),
            optimizer=torch.optim.Adam,
            max_epochs=CNN_MAX_EPOCHS,
            callbacks=[
                EpochScoring(
                    scoring="f1_macro",
                    name="valid_f1",
                    lower_is_better=False,
                ),
            ],
            # Internal 80/20 train valid split
            device="cuda",
            verbose=False,
        ),
    )

    clf.fit(X_img_train, y_img_train)

    # Avoid overfitting by evaluating internal 20% validation
    return np.mean([e.history[-1, "valid_loss"] for e in clf.estimators_])


# 3. Create a study object and optimize the objective function.
cnn_study = optuna.create_study(
    direction="minimize",
    study_name="cnn",
    storage=f"sqlite:///{model_dir}/optuna.db",
    load_if_exists=True,
)

cnn_study.optimize(
    objective, n_trials=N_TRIALS, n_jobs=1, show_progress_bar=True, gc_after_trial=False
)

[I 2021-11-02 13:19:12,410] A new study created in RDB with name: cnn


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2021-11-02 13:26:22,779] Trial 0 finished with value: 1.787541131258011 and parameters: {'module__out_channels': 7, 'module__f_units_l0': 165, 'module__f_units_l1': 28}. Best is trial 0 with value: 1.787541131258011.
[I 2021-11-02 13:34:16,771] Trial 1 finished with value: 1.6368818435296415 and parameters: {'module__out_channels': 8, 'module__f_units_l0': 286, 'module__f_units_l1': 80}. Best is trial 1 with value: 1.6368818435296415.
[I 2021-11-02 13:45:02,404] Trial 2 finished with value: 1.5149233328104021 and parameters: {'module__out_channels': 13, 'module__f_units_l0': 236, 'module__f_units_l1': 64}. Best is trial 2 with value: 1.5149233328104021.
[I 2021-11-02 13:49:40,576] Trial 3 finished with value: 1.3019081979990004 and parameters: {'module__out_channels': 2, 'module__f_units_l0': 68, 'module__f_units_l1': 67}. Best is trial 3 with value: 1.3019081979990004.
[I 2021-11-02 14:00:09,168] Trial 4 finished with value: 1.3942262022197247 and parameters: {'module__out_channels

# Train Models

In [16]:
rf_model = OneVsRestClassifier(
    make_pipeline(
        StandardScaler(),
        XGBRFClassifier(
            **rf_study.best_params,
            use_label_encoder=False,
            eval_metric="logloss",  # set to hide warning
        ),
    )
)

rf_model.fit(X_train.numpy(), y_train.numpy())
pickle.dump(rf_model, open(f"{model_dir}/rf_model.pkl", "wb"))

In [17]:
svc_model = OneVsRestClassifier(
    make_pipeline(
        StandardScaler(),
        SVC(
            **svc_study.best_params,
            probability=True,
        ),
    )
)

svc_model.fit(X_train.numpy(), y_train.numpy())
pickle.dump(svc_model, open(f"{model_dir}/svc_model.pkl", "wb"))

In [18]:
# Train on 80%, 20% validation for early stopping
nn_model = OneVsRestClassifier(
    make_pipeline(
        StandardScaler(),
        NeuralNetClassifier(
            module=FCModule,
            **nn_study.best_params,
            callbacks=[EarlyStopping(monitor="valid_loss", patience=20)],
            optimizer=torch.optim.Adam,
            max_epochs=NN_MAX_EPOCHS,
            device="cuda",
            verbose=False,
        ),
    ),
)

nn_model.fit(X_train, y_train)
pickle.dump(nn_model, open(f"{model_dir}/nn_model.pkl", "wb"))

In [19]:
# Train on 80%, 20% validation for early stopping
cnn_model = OneVsRestClassifier(
    NeuralNetClassifier(
        module=SpotsBinaryModule,
        **cnn_study.best_params,
        module__n_conv_layers=2,
        module__in_dim=64,
        module__kernel_size=3,
        callbacks=[EarlyStopping(monitor="valid_loss", patience=20)],
        optimizer=torch.optim.Adam,
        max_epochs=CNN_MAX_EPOCHS,
        device="cuda",
        verbose=False,
    ),
)

cnn_model.fit(X_img_train, y_img_train)
pickle.dump(cnn_model, open(f"{model_dir}/cnn_model.pkl", "wb"))